In [2]:
import json
import os
import pandas as pd
import boto3

import torch
import create_test_data


had to downgrade conda to python=3.9 like:
conda create -n myenv python=3.9

In [3]:
t_cd = create_test_data.transcript_pickle_to_pd()

# Testing Embedding

In [7]:
t_cd = create_test_data.transcript_pickle_to_pd()
t_cd_small=t_cd[0:5]
t_cd_small

,transcript_id,sentences
0,d0a7e5864959,"[And older woman Jocasta Zamarripa., Shortly, ..."
1,e9a7a8ac9081,"[Meeting., My name is Cavalier Johnson., I'm c..."
2,694b0e5b01a7,[Joining you this morning is Vice Chair Alderm...
3,ad734a167e5a,"[Our first meeting of 2020, the Judiciary and ..."
4,fe845b99f32e,"[Alderman Hamilton., Here., Kovach., Here., Ba..."


In [4]:
%ls

README.md                       environment.yml
__pycache__/                    environment_op.yml
ami-and-icsi-corpora-master/    lambda_function.py
ami_isci_json_loading.ipynb     solbiati_experiments.ipynb
ami_isci_json_loading_v2.ipynb  t_cd_small.csv
aws_instructions.txt            test_data_create.ipynb
batch_function.py               testing.ipynb
create_test_data.py             testing_aws.ipynb
data_for_s3.pickle              transcript_42.csv
data_to_s3.py                   transcripts.pickle
download_data.ipynb             unsupervised_topic_segmentation/


## Save Data to S3

## Create Lambda Function

In [4]:
aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')

In [12]:
with open('embed_lambda.zip', 'rb') as f:
    lambda_zip = f.read()

try:
    # If function hasn't yet been created, create it
    response = aws_lambda.create_function(
        FunctionName='embed_lambda',
        Runtime='python3.9',
        Role=role['Role']['Arn'],
        Handler='lambda_function.lambda_handler',
        Code=dict(ZipFile=lambda_zip),
        Timeout=300
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip
    # file contents
    response = aws_lambda.update_function_code(
        FunctionName='embed_lambda',
        ZipFile=lambda_zip
        )

lambda_arn = response['FunctionArn']
print(lambda_arn)

arn:aws:lambda:us-east-1:727739772368:function:embed_lambda


In [8]:
list(t_cd_small['sentences'])[0][0]

'And older woman Jocasta Zamarripa.'

In [9]:

r = aws_lambda.invoke(FunctionName='embed_lambda', #replace with name of the function
                    InvocationType='RequestResponse',
                    Payload=json.dumps(list(t_cd_small['sentences'])))
response=json.loads(r['Payload'].read()) # print out response
print(response)
    

    

{'errorMessage': "Unable to import module 'lambda_function': No module named 'torch'", 'errorType': 'Runtime.ImportModuleError', 'requestId': '303338f6-bd9b-4ec0-8326-e146fc39e4d3', 'stackTrace': []}


In [10]:
from platform import python_version
print(python_version())

3.10.6


{'errorMessage': "Unable to import module 'lambda_function': No module named 'torch'", 'errorType': 'Runtime.ImportModuleError', 'requestId': '0cdaaaf8-9694-46c5-8cd2-be0604e9f6dd', 'stackTrace': []}

change lamda
take list of lists 
and return list of lists


## Create Queue

In [28]:
# Create a batch client
batch = boto3.client('batch')

# Define the input and output S3 bucket and file paths
input_bucket = 'decoding-democracy-embded'
input_key = 'unembedded.csv'
output_bucket = 'decoding-democracy-embded'
output_key = 'embedded.csv'

# Define the event payload
event = {
    'input_bucket': input_bucket,
    'input_key': input_key,
    'output_bucket': output_bucket,
    'output_key': output_key
}

# Define the job definition
job_definition = 'decoding-democracy-env'

# Define the job name
job_name = 'decoding_democracy_job'

# Define the job queue
job_queue = 'decoding-democracy-queue'

In [29]:
# Submit the job
response = batch.submit_job(
    jobName=job_name,
    jobQueue=job_queue,
    jobDefinition=job_definition,
    parameters=event
)

# Print the job ID
print('Submitted batch job:', response['jobId'])

Submitted batch job: 8ffabaaa-eaaa-4538-8a1b-2de1d7c7cb95
